In [1]:
import time
start_time = time.time()
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider()

In [2]:
# Standard imports
import numpy as np
import pandas as pd
import time
import warnings

# Qiskit 
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes, EfficientSU2
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.circuit.library import QNNCircuit
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, ADAM, SLSQP, AQGD
from qiskit_algorithms.utils import algorithm_globals

# Visualization libraries
from matplotlib import pyplot as plt
import seaborn as sns
from qiskit.visualization import plot_histogram
from IPython.display import clear_output

# Machine learning and preprocessing
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.exceptions import ConvergenceWarning
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [10]:
from sklearn.datasets import load_iris
iris_data = load_iris()
features = iris_data.data
labels = iris_data.target
features = MinMaxScaler().fit_transform(features)
algorithm_globals.random_seed = 4701

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, train_size = 0.8, random_state = algorithm_globals.random_seed)
num_features = features.shape[1]

In [11]:
sampler = Sampler()


In [12]:
objective_func_vals = []
def callback_graph(eval_count, parameters, mean, std):
    objective_func_vals.append(mean)
    clear_output(wait=True)
    plt.title('Objective function value against Iteration')
    plt.xlabel('Iteration')
    plt.ylabel('Objective function value')
    plt.plot(objective_func_vals)
    plt.show()

# Setup combinations
feature_dimension = 4
combinations = [
    {"optimizer": optimizer, "ansatz": ansatz(feature_dimension, reps=reps)}
    for optimizer in [COBYLA(maxiter=100), SLSQP(maxiter=100)]
    for ansatz in [RealAmplitudes, EfficientSU2]
    for reps in range(1, 5)
]

results = []

# Iterate over combinations and train VQC
for combo in combinations:
    optimizer = combo["optimizer"]
    ansatz = combo["ansatz"]
    feature_map = ZZFeatureMap(feature_dimension=feature_dimension, reps=1)

    # Instantiate VQC
    vqc = VQC(feature_map=feature_map,
              ansatz=ansatz,
              optimizer=optimizer,
              callback=callback_graph,
              sampler=sampler)

    # Fit model and measure time
    start_time = time.time()
    vqc.fit(train_features, train_labels)
    elapsed_time = time.time() - start_time


    score = vqc.score(test_features, test_labels) # Example evaluation

    # Store results
    results.append({
        "optimizer": type(optimizer).__name__,
        "ansatz": type(ansatz).__name__,
        "ansatz_reps": ansatz.reps,
        "training_time": elapsed_time,
        # "score": score  # Include evaluation score in results
    })

# Output or further process your results
for result in results:
    print(result)

TypeError: callback_graph() missing 2 required positional arguments: 'mean' and 'std'